<center>
    <img src="../imagens/logo_APL.png" width="300" alt="APL logo"  />
</center>

# Comparação de Modelos de Classificação

**Bem vindo!** Neste material você usará os 3 classificadores vistos (SVM,  KNN e Árvore de Decisão) em três conjuntos de dados sintéticos. Ao final, espera-se que você seja capaz de entender como utilizar métricas de desempenho para fazer uma análise comparativa entre diferentes algoritimos.

<h2>Conteúdo:</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li> Introdução </li>
    <li> Criação de Dados Sintéticos </li> 
    <li> Escolha dos Classificadores </li>
    <li> Procedimento Experimental </li>  
    <li> Discussão </li>  
    <li> Considerações Finais </li>
</ul>
</div>

<hr>

## Introdução

Já vimos  como a biblioteca `scikit-learn` facilita as etapas de treinamento e avaliação dos modelos. 

Para a classificação, a acurácia é uma das medidas básicas de desempenho. Essa métrica indica o percentual de classes que foram corretamente classificadas. Ela pode ser calculada da seguinte forma:

<center>
    <img src="../imagens/acuracia-1.png" width="300" alt="Acurácia"  />
</center>

Neste material você usará a acurácia para análise comparativa do desempenho dos classificadores estudados ao longo do nosso curso. 

Vamos carregar as bibliotecas:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


## Criação dos Dados Sintéticos

Serão criados 3 conjuntos de dados com o auxílio das funções `make_moons`, `make_circles` e `make_classification` da biblioteca `scikit-learn`. Essas funções são preprogramadas para gerar dados em formatos específicos que podem ser usados para testes de modelos simples. 

Em cada dataset os dados serão divididos em duas classes. Trata-se, portanto, de um problema de classificação binária.

Finalmente, cada dataset é formado pela `tupla (X, y)`, em que:
- `X` representa os regressores (a matriz de dados de entrada); e
- `y` a variável alvo (ou vetor de saída).  

## Escolha dos Classificadores

Incialmente, vamos definir os nomes para identificarmos os algoritmos:

In [ ]:
names = ["KNN", "SVM", "Árvore de Decisão"]

Vamos agora definir os construtores dos algoritmos que vamos usar:

In [ ]:
classifiers = [
    KNeighborsClassifier(), 
    SVC(), 
    DecisionTreeClassifier()]

Agora vamos definir os nossos datasets:

In [ ]:
datasets = [make_classification(n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1),
            make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.3, factor=0.5, random_state=1)
            ]

## Procedimento Experimental

O processo se dará da seguinte forma:

- Será feito um laço para passar por cada um dos 3 dataset criados, nessa etapa os dados serão separados entre conjunto de dados e conjunto de treino.
- Dentro do primeiro laço haverá um segundo que irá passar pelos modelos de classficação, aqui serão feitos os treinamentos.

Com isso, cada modelo passará por cada dataset em sequência.

A visualização será dada pela função <code>scatter</code>, as classes serão identificadas por cores.

Ao final, será possível ver também os limites (ou **regiões de classificação**) que cada modelo gerou para cada dataset.

In [ ]:
figure = plt.figure(figsize=(27, 9))
i = 1
# Laço em cima dos datasets
for ds_cnt, ds in enumerate(datasets):
    # Separação dos datasets em treino/teste
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

    # Plotando os datasets
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Dados de entrada", fontsize=30)
    # Plota os dados de teste
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
    # Plota os dados de treinamento
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    
    i += 1

    # Laço em cima dos modelos
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # [Curiosidade] Essa parte está fazendo a superfície de decisão, onde 
        # é possível ver como cada modelo está fazendo a classificação dos dados.
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plota os dados de treino
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
        # Plota os dados de teste
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, edgecolors='w', alpha=0.5)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name, fontsize=30)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'), 
                size=15, horizontalalignment='right', fontsize=25)
        i += 1

plt.tight_layout()
plt.show()

## Discussão

Você pode observar a métrica de desempenho no canto inferior. Pode-se concluir, portanto, que os algoritmos obtiverem desempenho equivalente para o primeiro dataset. O algoritmo KNN obteve desempenho melhor na segunda base de dados, enquanto o algoritmo SVM na terceira.

## Considerações Finais

Neste material vimos como a biblioteca `scikit-learn` possui métodos projetados para auxiliar o programador ao longo do processo de *Machine Learning*. Fizemos uma análise comparativa dos 3 classificadores estudadosa ao longo do nosso curso.

<hr>

## Direitos Autorais

[APL Data Intelligence](https://linktr.ee/APLdataintelligence)&#8482;  2021. Este notebook Python e seu código fonte estão liberados sob os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).